In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd

# === Load Data ===
npy_data = np.load("radial_pos_enc.npy")
pt_data = torch.load("radial_pos_enc.pt")

# === Print Shapes ===
print("NPY shape:", npy_data.shape)
print("PT shape:", pt_data.shape)

# === Display First Ring Vector from Batch 1 ===
print("\nNPY - Batch 1, Ring 1 vector (first 10 dims):")
print(npy_data[0, 0][:10])

print("\nPT - Batch 1, Ring 1 vector (first 10 dims):")
print(pt_data[0, 0][:10])

# === Visualize All 4 Ring Embeddings ===
plt.figure(figsize=(10, 4))
for ring in range(npy_data.shape[1]):
    plt.plot(npy_data[0, ring], label=f"Ring {ring+1}")
plt.title("Learnable Positional Embeddings (Batch 1)")
plt.xlabel("Embedding Dimension")
plt.ylabel("Value")
plt.legend()
plt.tight_layout()
plt.show()

# === Display Full Ring Vectors in Table ===
df = pd.DataFrame(npy_data[0], index=[f"Ring {i+1}" for i in range(4)])
df.columns = [f"D{i}" for i in range(npy_data.shape[2])]
df

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import os

# Paths to the output files
output_dir = "output"
tokens_9d_path = os.path.join(output_dir, "radial_tokens_9D.pt")
tokens_192d_path = os.path.join(output_dir, "radial_tokens_192D.pt")
projection_weights_path = os.path.join(output_dir, "radial_tokens_projection_weights.pt")
metadata_path = os.path.join(output_dir, "radial_tokens_metadata.pt")

# Load tensors
tokens_9d = torch.load(tokens_9d_path)
tokens_192d = torch.load(tokens_192d_path)

print(f"✔ Loaded: 9D → {tokens_9d.shape}, 192D → {tokens_192d.shape}")

tokens_9d_np = tokens_9d.squeeze(0).detach().numpy()
tokens_192d_np = tokens_192d.squeeze(0).detach().numpy()

# Display 9D features
df_9d = pd.DataFrame(tokens_9d_np,
                     columns=[f"Feature_{i+1}" for i in range(9)],
                     index=[f"Ring_{i+1}" for i in range(4)])

print("\n🔹 9D Feature Vectors (Mean, Std, Median per Ring):")
display(df_9d)

# Display 192D projected embeddings
df_192d = pd.DataFrame(tokens_192d_np,
                       columns=[f"Dim_{i+1}" for i in range(192)],
                       index=[f"Ring_{i+1}" for i in range(4)])

print("\n🔹 192D Projected Embeddings:")
display(df_192d)

# Visualize 192D embeddings
plt.figure(figsize=(10, 5))
for i in range(4):
    plt.plot(tokens_192d_np[i], label=f"Ring {i+1}")
plt.title("192D Token Embeddings per Ring")
plt.xlabel("Embedding Dimension")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Load projection layer weights
if os.path.exists(projection_weights_path):
    projector_state = torch.load(projection_weights_path)
    weight = projector_state['proj.weight'].detach().cpu().numpy()
    bias = projector_state['proj.bias'].detach().cpu().numpy()

    print("\n📦 Projection Layer Weights [192 × 9]:", weight.shape)
    df_weights = pd.DataFrame(weight, columns=[f"Input_{i+1}" for i in range(9)])
    display(df_weights.head())

    print("\n📦 Projection Layer Bias [192]:", bias.shape)
    print("Preview Bias Vector:", bias[:10])
else:
    print("\n⚠️ No projection weight file found.")

# Load metadata
if os.path.exists(metadata_path):
    metadata = torch.load(metadata_path)
    print("\n📌 Metadata Contents:")
    for key, value in metadata.items():
        if isinstance(value, torch.Tensor):
            print(f"{key}: Tensor → {value.shape}")
        else:
            print(f"{key}: {value}")
else:
    print("\n⚠️ No metadata file found.")